In [1]:
#r "nuget:Voice100"
#r "nuget:Voice100.Native"
#r "nuget:Microsoft.ML.OnnxRuntime"
#r "nuget:Voice100.Interactive, 0.2.2-dev"

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed Packages Microsoft.ML.OnnxRuntime, 1.11.0 Voice100, 1.2.1 Voice100.Native, 0.3.0

Installed Packages Voice100.Interactive, 0.2.2-dev

Loading extensions from `Voice100.Interactive.dll`

In [14]:
using System.Net.Http;
using Voice100;
using Voice100.Interactive;

In [5]:
string cacheDirectoryPath = "./cache";
bool useMultiTaskTTS = true;

In [6]:
async Task<SpeechSynthesizer> BuildSpeechSynthesizerAsync(bool useMultiTaskTTS)
{
    string alignModelPath;
    string audioModelPath;
    using (var httpClient = new HttpClient())
    {
        var downloader = new ModelDownloader(httpClient, cacheDirectoryPath);
        alignModelPath = await downloader.MayDownloadAsync(
            "ttsalign_en_conv_base-20210808.onnx",
            "https://github.com/kaiidams/voice100-runtime/releases/download/v0.1/ttsalign_en_conv_base-20210808.onnx",
            "D87B80B2C9CC96AC7A4C89C979C62FA3C18BACB381C3C1A3F624A33496DD1FC8");
        if (useMultiTaskTTS)
        {
            audioModelPath = await downloader.MayDownloadAsync(
                "ttsaudio_en_mt_conv_base-20220316.onnx",
                "https://github.com/kaiidams/voice100-runtime/releases/download/v1.2.0/ttsaudio_en_mt_conv_base-20220316.onnx",
                "5d0f426509bb662deab3ca9cf964f68dbaf2a30b55e653205c98eaad63978468");
        }
        else
        {
            audioModelPath = await downloader.MayDownloadAsync(
                "ttsaudio_en_conv_base-20220107.onnx",
                "https://github.com/kaiidams/voice100-runtime/releases/download/v1.0.1/ttsaudio_en_conv_base-20220107.onnx",
                "A20FEC366D1A4856006BBF7CFAC7D989EF02B0C1AF676C0B5E6F318751325A2F");
        }
    }
    return new SpeechSynthesizer(alignModelPath, audioModelPath, useMultiTaskTTS);
}

In [7]:
var synthesizer = await BuildSpeechSynthesizerAsync(useMultiTaskTTS);

Using cached `ttsalign_en_conv_base-20210808.onnx'.
Using cached `ttsaudio_en_mt_conv_base-20220316.onnx'.


In [23]:
var waveform = synthesizer.Speak("When you take actions across github, we'll provide links to that activity here.");

In [24]:
new Audio(MemoryMarshal.Cast<byte, short>(waveform).ToArray(), 16000)